In [ ]:
## This is final check file to make sure math is working correctly and is similar to originally tested models from team in python
#from ShelterData_ML_Final.ipynb

In [ ]:
### Update  code here to test different csvs, we're using our test data set
import shutil; shutil.copyfile('final_pipeline_prediction_sample_Test.csv', 'final_pipeline_prediction.csv')
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score,
    recall_score, f1_score, roc_auc_score, roc_curve, classification_report)
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
import matplotlib.pyplot as plt



In [ ]:
#Checking Adoption Prediciton first


#load
df_truth = pd.read_csv('df_cat_dog_harmonized.csv') # This is ground truth file
df_pred = pd.read_csv('final_pipeline_prediction.csv')

#Merge
df = pd.merge(df_truth[['primary_key','outcome_type_harmonized', 'outcome_type_harmonized_grouped', 'stay_length_days']], 
              df_pred[['primary_key','predicted_label', 'predicted_proba','stay_length_predicted', 'non_adopted_label']], 
              on='primary_key')

#Binarize ground truth: adopted = 1, else 0

df['actual_adopt'] = df['outcome_type_harmonized_grouped'].apply(lambda x: 1 if str(x).strip().lower() == 'adopted' else 0)

# Metrics

print("Confusion Matrix:")
print(confusion_matrix(df['actual_adopt'], df['predicted_label']))

print("\nClassification Report:")
print(classification_report(df['actual_adopt'], df['predicted_label'], labels=[0, 1], target_names=["Not Adopted", "Adopted"]))


print(f"Accuracy: {accuracy_score(df['actual_adopt'], df['predicted_label']):.4f}")
print(f"Precision: {precision_score(df['actual_adopt'], df['predicted_label']):.4f}")
print(f"Recall: {recall_score(df['actual_adopt'], df['predicted_label']):.4f}")
print(f"F1 Score: {f1_score(df['actual_adopt'], df['predicted_label']):.4f}")


# AUC

if 'predicted_proba' in df.columns:
    auc = roc_auc_score(df['actual_adopt'], df['predicted_proba'])
    print(f"AUC: {auc:.4f}")

#Plot ROC Curve
    fpr, tpr, _ = roc_curve(df['actual_adopt'], df['predicted_proba'])
    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel("False Positive Rate")


In [ ]:
df.head()

In [ ]:
#Weighted Accuracy is good and AUC is good, so we keep model as is.

In [ ]:
#Checking predicted stay lenght but only for adopted 

#  only include rows where predicted_proba >= 0.5
df_filtered = df[df['predicted_proba'] >= 0.5].copy()

# Drop rows with missing values in either column
df_filtered = df_filtered.dropna(subset=['stay_length_days', 'stay_length_predicted'])

# Evaluate metrics
r2 = r2_score(df_filtered['stay_length_days'], df_filtered['stay_length_predicted'])
mse = mean_squared_error(df_filtered['stay_length_days'], df_filtered['stay_length_predicted'])
mae = mean_absolute_error(df_filtered['stay_length_days'], df_filtered['stay_length_predicted'])

print(f"Filtered Rows: {len(df_filtered)}")
print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")


In [ ]:
#R2 is pretty similar to what we expected, we ended up dropping this.
#Now we evlauate 

In [ ]:
# Clean and map ground truth to integer labels
label_map = {
    "foster": 0,
    "foster to adopt": 0,
    "rtf": 0,
    "rescue": 1,
    "return to rescue": 1,
    "return to owner": 2
}
df['outcome_type_harmonized'] = df['outcome_type_harmonized'].str.lower().str.strip()
df['actual_label'] = df['outcome_type_harmonized'].map(label_map)
df['predicted_label'] = df['non_adopted_label'].astype(int)

In [ ]:
df

In [ ]:
#Lastly, we'll predict multiclass outcome


# Drop rows with unknown classes 
df = df.dropna(subset=['actual_label'])

# Keep only rows where predicted adoption confidence is low
df = df[df['predicted_proba'] < 0.5]


# Evaluation
print("Unique ground truth labels:")
print(df['outcome_type_harmonized'].unique())

print("\nValue counts after mapping:")
print(df['actual_label'].value_counts(dropna=False))


print("\nConfusion Matrix:")
print(confusion_matrix(df['actual_label'], df['predicted_label']))

print("\nClassification Report:")
print(classification_report(df['actual_label'], df['predicted_label'],
                            target_names=['foster', 'rescue', 'return_to_owner']))


